In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType


off_train = spark.read.csv('ccf_offline_stage1_train.csv', header=True, inferSchema=True)
off_test = spark.read.csv('ccf_offline_stage1_test_revised.csv', header = True, inferSchema=True)
on_train = spark.read.csv('ccf_online_stage1_train.csv', header = True, inferSchema = True)

In [2]:
dataset3 = off_test
feature3 = off_train.filter(((off_train.Date>='20160315')&(off_train.Date<='20160630'))\
                           |((off_train.Date=='null') & (off_train.Date_received>='20160315')&(off_train.Date_received<='20160630')))\
                        .select('*')
dataset2 = off_train.filter((off_train.Date_received>='20160515')&(off_train.Date_received<='20160615')).select('*')
feature2 = off_train.filter((off_train.Date>='20160201')&(off_train.Date<='20160514')|((off_train.Date=='null')&(off_train.Date_received>='20160201')&(off_train.Date_received<='20160514'))).select('*')
dataset1 = off_train.filter((off_train.Date_received>='20160414')&(off_train.Date_received<='20160514')).select('*')
feature1 = off_train.filter((off_train.Date>='20160101')&(off_train.Date<='20160413')|((off_train.Date=='null')&(off_train.Date_received>='20160101')&(off_train.Date_received<='20160413'))).select('*')

In [3]:
dataset3.show()
dataset2.show()
dataset1.show()
feature3.show()

+-------+-----------+---------+-------------+--------+-------------+
|User_id|Merchant_id|Coupon_id|Discount_rate|Distance|Date_received|
+-------+-----------+---------+-------------+--------+-------------+
|4129537|        450|     9983|         30:5|       1|     20160712|
|6949378|       1300|     3429|         30:5|    null|     20160706|
|2166529|       7113|     6928|       200:20|       5|     20160727|
|2166529|       7113|     1808|       100:10|       5|     20160727|
|6172162|       7605|     6500|         30:1|       2|     20160708|
|4005121|        450|     9983|         30:5|       0|     20160706|
|4347394|        450|     9983|         30:5|       0|     20160716|
|3094273|        760|    13602|         30:5|       1|     20160727|
|5139970|        450|     9983|         30:5|      10|     20160729|
|3237121|        760|    13602|         30:5|       1|     20160703|
|6224386|        450|     9983|         30:5|       3|     20160716|
|6488578|        760|    13602|   

In [29]:
###5. other feature:###

#for dataset3
t = dataset3.select('User_id')
t = t.select('User_id').groupBy('User_id').count().withColumnRenamed('count', 'Month_Receive_all_coupon_count')

#print(t.orderBy('User_id').show())

In [30]:
t1 = dataset3.select('User_id','Coupon_id')
t1 = t1.select('User_id','Coupon_id').groupBy('User_id','Coupon_id').count().withColumnRenamed('Count', 'Month_Receive_same_coupon_count')

#print(t1.orderBy('User_id').show())

In [31]:
t2 = dataset3.select('User_id', 'Coupon_id', 'Date_received')

In [32]:
from pyspark.sql import functions as F
t2 = t2.groupby(['User_id', 'Coupon_id']).agg(F.max("Date_received"),F.min("Date_received"))
t2 = t2.withColumnRenamed('max(Date_received)','Max_date_received').withColumnRenamed('Min(Date_received)','Min_date_received')
t2.show()

+-------+---------+-----------------+-----------------+
|User_id|Coupon_id|Max_date_received|Min_date_received|
+-------+---------+-----------------+-----------------+
|3649795|    10418|         20160701|         20160701|
|3025159|    10438|         20160701|         20160701|
|2296075|    13602|         20160728|         20160728|
|7346734|     4405|         20160701|         20160701|
|2970925|     5178|         20160728|         20160711|
|2746177|    13602|         20160704|         20160704|
|5632654|     4138|         20160716|         20160716|
|5406394|    13602|         20160705|         20160705|
|4451026|     9983|         20160711|         20160711|
|2891131|     3429|         20160724|         20160724|
|3032473|     8306|         20160715|         20160715|
|2447773|    12807|         20160703|         20160703|
|6210370|     2978|         20160704|         20160704|
|4541266|     9983|         20160719|         20160719|
|3618223|    11799|         20160712|         20

In [ ]:
t3 = dataset3.select('User_id','Coupon_id','Date_received')
t3 = t3.join(t2, ['User_id','Coupon_id'])
t3 = t3.withColumn('Month_same_coupon_lastone', col('Max_date_received') - col('date_received'))
t3 = t3.withColumn('Month_same_coupon_firstone',col('date_received') - col('Min_date_received'))                   

#print(t3.orderBy('User_id').show())

In [ ]:
equally1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(1).alias('feat1')).filter('Month_same_coupon_lastone == 0').drop('Month_same_coupon_lastone')
greater1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(0).alias('feat1')).filter('Month_same_coupon_lastone > 0').drop('Month_same_coupon_lastone')
u1 = equally1.union(greater1).withColumnRenamed('feat1', 'Month_same_coupon_lastone')
u1 = u1.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone')
t3 = t3.drop('Month_same_coupon_lastone')

equally2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(1).alias('feat2')).filter('Month_same_coupon_firstone == 0').drop('Month_same_coupon_firstone')
greater2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(0).alias('feat2')).filter('Month_same_coupon_firstone > 0').drop('Month_same_coupon_firstone')
u2 = equally2.union(greater2).withColumnRenamed('feat2', 'Month_same_coupon_firstone')
u2 = u2.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone')
t3 = t3.drop('Month_same_coupon_firstone')

t3 = t3.join(u1, ['User_id','Coupon_id','Date_received'])
t3 = t3.join(u2, ['User_id','Coupon_id','Date_received'])
t3 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone','Month_same_coupon_firstone')

#print(t3.orderBy('User_id').show())
#print(u1.orderBy('User_id').show())
#print(u2.orderBy('User_id').show())

In [10]:
t4 = dataset3.select('User_id','Date_received')
t4 = t4.select('Date_received','User_id',).groupBy('User_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_all_coupon_count')

print(t4.orderBy('user_id').show())

+-------+-------------+----------------------------+
|User_id|Date_received|Day_Receive_all_coupon_count|
+-------+-------------+----------------------------+
|    209|     20160721|                           2|
|    215|     20160703|                           1|
|    316|     20160721|                           1|
|    417|     20160712|                           1|
|    432|     20160706|                           1|
|    448|     20160701|                           1|
|    448|     20160710|                           1|
|    448|     20160706|                           1|
|    448|     20160707|                           1|
|    448|     20160704|                           1|
|    452|     20160721|                           1|
|    452|     20160702|                           1|
|    470|     20160710|                           1|
|    569|     20160716|                           1|
|    728|     20160714|                           1|
|    736|     20160724|                       

In [11]:
t5 = dataset3.select('User_id','Coupon_id','Date_received')
t5 = t5.select('User_id','Coupon_id','Date_received').groupBy('User_id','Coupon_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_same_coupon_count')

print(t5.orderBy('user_id').show())

+-------+---------+-------------+-----------------------------+
|User_id|Coupon_id|Date_received|Day_Receive_same_coupon_count|
+-------+---------+-------------+-----------------------------+
|    209|      825|     20160721|                            1|
|    209|     7557|     20160721|                            1|
|    215|     5488|     20160703|                            1|
|    316|     3992|     20160721|                            1|
|    417|    12465|     20160712|                            1|
|    432|    10438|     20160706|                            1|
|    448|    13602|     20160701|                            1|
|    448|    10927|     20160706|                            1|
|    448|    10927|     20160710|                            1|
|    448|    10927|     20160704|                            1|
|    448|    10927|     20160707|                            1|
|    452|     6500|     20160702|                            1|
|    452|     3992|     20160721|       

In [12]:
t6 = dataset3.select('User_id','Coupon_id','Date_received')
t6 = t6.withColumn('Date_received',(col('Date_received').astype('string')))
t6 = t6.withColumn("Date_received",lit(to_date(t6.Date_received,"yyyyMMdd")))

t6.show()
t6

+-------+---------+-------------+
|User_id|Coupon_id|Date_received|
+-------+---------+-------------+
|4129537|     9983|   2016-07-12|
|6949378|     3429|   2016-07-06|
|2166529|     6928|   2016-07-27|
|2166529|     1808|   2016-07-27|
|6172162|     6500|   2016-07-08|
|4005121|     9983|   2016-07-06|
|4347394|     9983|   2016-07-16|
|3094273|    13602|   2016-07-27|
|5139970|     9983|   2016-07-29|
|3237121|    13602|   2016-07-03|
|6224386|     9983|   2016-07-16|
|6488578|    13602|   2016-07-12|
|4164865|     9983|   2016-07-03|
|4164865|     8059|   2016-07-06|
|5468674|     9983|   2016-07-13|
|6258178|     9144|   2016-07-06|
|3659521|     7341|   2016-07-27|
|3659521|    13181|   2016-07-17|
|3659521|    13602|   2016-07-18|
|7333378|    13602|   2016-07-04|
+-------+---------+-------------+
only showing top 20 rows



DataFrame[User_id: int, Coupon_id: int, Date_received: date]

In [13]:
t6 = dataset3.select('User_id','Coupon_id','Date_received')
t6 = t6.withColumn('Date_received',(col('Date_received').astype('string')))
t6 = t6.withColumn("Date_received",lit(to_date(t6.Date_received,"yyyyMMdd")))



t6 = t6.groupby('User_id', 'Coupon_id').sortBy('Date_received')
#.agg(collect_list('Date_received').alias('Date')).withColumnRenamed('Date','Dates')
print(t6.orderBy('User_id').show())
t6

AttributeError: 'GroupedData' object has no attribute 'sortBy'

In [ ]:
from pyspark.sql.functions import lit
t7 = dataset3.select('User_id','Coupon_id','Date_received')
t7 = t7.join(t6,['User_id','Coupon_id'])
t7 = t7.withColumn('Date_received',(col('Date_received').astype('string')))
t7 = t7.withColumn("Date_received",lit(to_date(t7.Date_received,"yyyyMMdd")))
#t7 = t7.withColumn('Day_gap_before',lit(datediff('Dates','Date_received')))
#t7 = t7.withColumn('single', lit('-'))
#t7 = t7.select('User_id','Coupon_id','Date_received','Dates', concat('Date_received','single','Dates')).withColumnRenamed('concat(Date_received, single, Dates)', 'Date_received_date')
print(t7.orderBy('User_id').show())
t7

In [ ]:
other_feature3 = t1.join(t, 'User_id')
other_feature3 = other_feature3.join(t3,['user_id','coupon_id'])
other_feature3 = other_feature3.join(t4, ['User_id','Date_received'])
other_feature3 = other_feature3.join(t5, ['User_id','Coupon_id','Date_received'])
#other_feature3 = other_feature3.join(t7, ['User_id','Coupon_id','Date_received'])
other_feature3.write.csv('other_feature3', header = True)

other_feature3.show()

In [ ]:
#for dataset2

In [3]:
t = dataset2.select('User_id')
t = t.select('User_id').groupBy('User_id').count().withColumnRenamed('count', 'Month_Receive_all_coupon_count')

#print(t.orderBy('User_id').show())

In [4]:
t1 = dataset2.select('User_id','Coupon_id')
t1 = t1.select('User_id','Coupon_id').groupBy('User_id','Coupon_id').count().withColumnRenamed('Count', 'Month_Receive_same_coupon_count')

#print(t1.orderBy('User_id').show())

In [5]:
t2 = dataset2.select('User_id', 'Coupon_id', 'Date_received')

In [6]:
from pyspark.sql import functions as F
t2 = t2.groupby(['User_id', 'Coupon_id']).agg(F.max("Date_received"),F.min("Date_received"))
t2 = t2.withColumnRenamed('max(Date_received)','Max_date_received').withColumnRenamed('Min(Date_received)','Min_date_received')
t2.show()

+-------+---------+-----------------+-----------------+
|User_id|Coupon_id|Max_date_received|Min_date_received|
+-------+---------+-----------------+-----------------+
|   1797|     3312|         20160524|         20160524|
|   2574|    13208|         20160611|         20160611|
|   7099|     1141|         20160518|         20160518|
|   8094|     8067|         20160519|         20160519|
|  10608|     1532|         20160531|         20160531|
|  17068|     5317|         20160515|         20160515|
|  38165|    10706|         20160521|         20160521|
|  40712|     9296|         20160530|         20160530|
|  44316|     2418|         20160611|         20160611|
|  55266|     5899|         20160518|         20160518|
|  58581|    11195|         20160525|         20160525|
|  72805|     5899|         20160515|         20160515|
|  79512|     2705|         20160523|         20160523|
|  80170|     8192|         20160517|         20160517|
|  85010|     2366|         20160527|         20

In [7]:
t3 = dataset2.select('User_id','Coupon_id','Date_received')
t3 = t3.join(t2, ['User_id','Coupon_id'])
t3 = t3.withColumn('Month_same_coupon_lastone', col('Max_date_received') - col('date_received'))
t3 = t3.withColumn('Month_same_coupon_firstone',col('date_received') - col('Min_date_received'))                   

print(t3.orderBy('User_id').show())

+-------+---------+-------------+-----------------+-----------------+-------------------------+--------------------------+
|User_id|Coupon_id|Date_received|Max_date_received|Min_date_received|Month_same_coupon_lastone|Month_same_coupon_firstone|
+-------+---------+-------------+-----------------+-----------------+-------------------------+--------------------------+
|      4|     2902|     20160607|         20160607|         20160607|                      0.0|                       0.0|
|    165|     7571|     20160525|         20160525|         20160525|                      0.0|                       0.0|
|    166|     9261|     20160525|         20160525|         20160525|                      0.0|                       0.0|
|    215|     8944|     20160524|         20160524|         20160524|                      0.0|                       0.0|
|    236|    11002|     20160528|         20160528|         20160528|                      0.0|                       0.0|
|    238|     24

In [8]:
equally1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(1).alias('feat1')).filter('Month_same_coupon_lastone == 0').drop('Month_same_coupon_lastone')
greater1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(0).alias('feat1')).filter('Month_same_coupon_lastone > 0').drop('Month_same_coupon_lastone')
u1 = equally1.union(greater1).withColumnRenamed('feat1', 'Month_same_coupon_lastone')
u1 = u1.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone')
t3 = t3.drop('Month_same_coupon_lastone')

equally2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(1).alias('feat2')).filter('Month_same_coupon_firstone == 0').drop('Month_same_coupon_firstone')
greater2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(0).alias('feat2')).filter('Month_same_coupon_firstone > 0').drop('Month_same_coupon_firstone')
u2 = equally2.union(greater2).withColumnRenamed('feat2', 'Month_same_coupon_firstone')
u2 = u2.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone')
t3 = t3.drop('Month_same_coupon_firstone')

t3 = t3.join(u1, ['User_id','Coupon_id','Date_received'])
t3 = t3.join(u2, ['User_id','Coupon_id','Date_received'])
t3 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone','Month_same_coupon_firstone')

print(t3.orderBy('User_id').show())
#print(u1.orderBy('User_id').show())
#print(u2.orderBy('User_id').show())

+-------+---------+-------------+-------------------------+--------------------------+
|User_id|Coupon_id|Date_received|Month_same_coupon_lastone|Month_same_coupon_firstone|
+-------+---------+-------------+-------------------------+--------------------------+
|      4|     2902|     20160607|                        1|                         1|
|    165|     7571|     20160525|                        1|                         1|
|    166|     9261|     20160525|                        1|                         1|
|    215|     8944|     20160524|                        1|                         1|
|    236|    11002|     20160528|                        1|                         1|
|    238|     2418|     20160608|                        1|                         1|
|    239|     1255|     20160614|                        1|                         1|
|    239|     9762|     20160614|                        1|                         1|
|    315|     1141|     20160518|          

In [9]:
t4 = dataset2.select('User_id','Date_received')
t4 = t4.select('Date_received','User_id',).groupBy('User_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_all_coupon_count')

#print(t4.orderBy('user_id').show())

In [10]:
t5 = dataset2.select('User_id','Coupon_id','Date_received')
t5 = t5.select('User_id','Coupon_id','Date_received').groupBy('User_id','Coupon_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_same_coupon_count')

#print(t5.orderBy('user_id').show())

In [11]:
t6 = dataset2.select('User_id','Coupon_id','Date_received')
t6 = t6.withColumn('Date_received',(col('Date_received').astype('string')))

In [ ]:
link = udf(lambda x: ':'.join(x))
t6 = t6.groupby('User_id', 'Coupon_id').agg(collect_list('Date_received').alias('Date_received')).withColumn('Date_received', link('Date_received'))
t6 = t6.withColumnRenamed('Date_received', 'Dates')

#t6.show()

In [ ]:
def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)

In [ ]:
from pyspark.sql.functions import lit
t7 = dataset2.select('User_id','Coupon_id','Date_received')
t7 = t7.join(t6,['User_id','Coupon_id'])
t7 = t7.withColumn('single', lit('-'))
t7 = t7.select('User_id','Coupon_id','Date_received','Dates', concat('Date_received','single','Dates')).withColumnRenamed('concat(Date_received, single, Dates)', 'Date_received_date')
#t7.withColumn('Day_gap_before', get_day_gap_before(col('date_received_date')))
#t7.withColumn('Day_gap_after', get_day_gap_after(col('date_received_date')))
#t7 = t7.select('User_id','Coupon_id','Date_received','Day_gap_before','Day_gap_after')

#t7.show()

In [12]:
other_feature2 = t1.join(t, 'User_id')
other_feature2 = other_feature2.join(t3,['user_id','coupon_id'])
other_feature2 = other_feature2.join(t4, ['User_id','Date_received'])
other_feature2 = other_feature2.join(t5, ['User_id','Coupon_id','Date_received'])
#other_feature2 = other_feature2.join(t7, ['User_id','Coupon_id','Date_received'])
other_feature2.write.csv('other_feature2', header = True)

other_feature2.show()

+-------+---------+-------------+-------------------------------+------------------------------+-------------------------+--------------------------+----------------------------+-----------------------------+
|User_id|Coupon_id|Date_received|Month_Receive_same_coupon_count|Month_Receive_all_coupon_count|Month_same_coupon_lastone|Month_same_coupon_firstone|Day_Receive_all_coupon_count|Day_Receive_same_coupon_count|
+-------+---------+-------------+-------------------------------+------------------------------+-------------------------+--------------------------+----------------------------+-----------------------------+
|  11170|    11873|     20160522|                              4|                            15|                        0|                         0|                           5|                            1|
|  11255|    13304|     20160521|                              1|                             1|                        1|                         1|               

In [ ]:
#for dataset1

In [4]:
t = dataset1.select('User_id')
t = t.select('User_id').groupBy('User_id').count().withColumnRenamed('count', 'Month_Receive_all_coupon_count')

#print(t.orderBy('User_id').show())

In [5]:
t1 = dataset1.select('User_id','Coupon_id')
t1 = t1.select('User_id','Coupon_id').groupBy('User_id','Coupon_id').count().withColumnRenamed('Count', 'Month_Receive_same_coupon_count')

#print(t1.orderBy('User_id').show())

In [6]:
t2 = dataset1.select('User_id', 'Coupon_id', 'Date_received')

In [7]:
from pyspark.sql import functions as F
t2 = t2.groupby(['User_id', 'Coupon_id']).agg(F.max("Date_received"),F.min("Date_received"))
t2 = t2.withColumnRenamed('max(Date_received)','Max_date_received').withColumnRenamed('Min(Date_received)','Min_date_received')
#t2.show()

In [8]:
t3 = dataset1.select('User_id','Coupon_id','Date_received')
t3 = t3.join(t2, ['User_id','Coupon_id'])
t3 = t3.withColumn('Month_same_coupon_lastone', col('Max_date_received') - col('date_received'))
t3 = t3.withColumn('Month_same_coupon_firstone',col('date_received') - col('Min_date_received'))                   
#t3.show()

In [9]:
equally1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(1).alias('feat1')).filter('Month_same_coupon_lastone == 0').drop('Month_same_coupon_lastone')
greater1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(0).alias('feat1')).filter('Month_same_coupon_lastone > 0').drop('Month_same_coupon_lastone')
u1 = equally1.union(greater1).withColumnRenamed('feat1', 'Month_same_coupon_lastone')
u1 = u1.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone')
t3 = t3.drop('Month_same_coupon_lastone')

equally2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(1).alias('feat2')).filter('Month_same_coupon_firstone == 0').drop('Month_same_coupon_firstone')
greater2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(0).alias('feat2')).filter('Month_same_coupon_firstone > 0').drop('Month_same_coupon_firstone')
u2 = equally2.union(greater2).withColumnRenamed('feat2', 'Month_same_coupon_firstone')
u2 = u2.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone')
t3 = t3.drop('Month_same_coupon_firstone')

t3 = t3.join(u1, ['User_id','Coupon_id','Date_received'])
t3 = t3.join(u2, ['User_id','Coupon_id','Date_received'])
t3 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone','Month_same_coupon_firstone')

print(t3.orderBy('User_id').show())
#print(u1.orderBy('User_id').show())
#print(u2.orderBy('User_id').show())

+-------+---------+-------------+-------------------------+--------------------------+
|User_id|Coupon_id|Date_received|Month_same_coupon_lastone|Month_same_coupon_firstone|
+-------+---------+-------------+-------------------------+--------------------------+
|    173|     7610|     20160414|                        1|                         1|
|    285|     1532|     20160501|                        1|                         1|
|    316|     8952|     20160430|                        1|                         1|
|    377|     2857|     20160512|                        1|                         1|
|    387|     7610|     20160421|                        1|                         1|
|    430|     9871|     20160419|                        1|                         1|
|    467|     9871|     20160417|                        1|                         1|
|    470|     9614|     20160424|                        1|                         1|
|    472|     3992|     20160504|          

In [10]:
t4 = dataset1.select('User_id','Date_received')
t4 = t4.select('Date_received','User_id',).groupBy('User_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_all_coupon_count')

#print(t4.orderBy('user_id').show())

In [11]:
t5 = dataset1.select('User_id','Coupon_id','Date_received')
t5 = t5.select('User_id','Coupon_id','Date_received').groupBy('User_id','Coupon_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_same_coupon_count')

#print(t5.orderBy('user_id').show())

In [ ]:
t6 = dataset1.select('User_id','Coupon_id','Date_received')
t6 = t6.withColumn('Date_received',(col('Date_received').astype('string')))

In [ ]:
link = udf(lambda x: ':'.join(x))
t6 = t6.groupby('User_id', 'Coupon_id').agg(collect_list('Date_received').alias('Date_received')).withColumn('Date_received', link('Date_received'))
t6 = t6.withColumnRenamed('Date_received', 'Dates')

#t6.show()

In [ ]:
def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)

In [ ]:
from pyspark.sql.functions import lit
t7 = dataset1.select('User_id','Coupon_id','Date_received')
t7 = t7.join(t6,['User_id','Coupon_id'])
t7 = t7.withColumn('single', lit('-'))
t7 = t7.select('User_id','Coupon_id','Date_received','Dates', concat('Date_received','single','Dates')).withColumnRenamed('concat(Date_received, single, Dates)', 'Date_received_date')
#t7.withColumn('Day_gap_before', get_day_gap_before(col('date_received_date')))
#t7.withColumn('Day_gap_after', get_day_gap_after(col('date_received_date')))
#t7 = t7.select('User_id','Coupon_id','Date_received','Day_gap_before','Day_gap_after')

#t7.show()

In [13]:
other_feature1 = t1.join(t, 'User_id')
other_feature1 = other_feature1.join(t3,['user_id','coupon_id'])
other_feature1 = other_feature1.join(t4, ['User_id','Date_received'])
other_feature1 = other_feature1.join(t5, ['User_id','Coupon_id','Date_received'])
#other_feature1 = other_feature3.join(t7, ['User_id','Coupon_id','Date_received'])
other_feature1.write.csv('other_feature1', header = True)

other_feature1.show()

+-------+---------+-------------+-------------------------------+------------------------------+-------------------------+--------------------------+----------------------------+-----------------------------+
|User_id|Coupon_id|Date_received|Month_Receive_same_coupon_count|Month_Receive_all_coupon_count|Month_same_coupon_lastone|Month_same_coupon_firstone|Day_Receive_all_coupon_count|Day_Receive_same_coupon_count|
+-------+---------+-------------+-------------------------------+------------------------------+-------------------------+--------------------------+----------------------------+-----------------------------+
|    377|     2857|     20160512|                              1|                             1|                        1|                         1|                           1|                            1|
|   5096|     5054|     20160426|                              1|                             1|                        1|                         1|               